- Starting point: https://quantdev.ssri.psu.edu/sites/qdev/files/topic_modeling_tutorial-Gutenberg-chapter_as_document_0.html

https://cran.r-project.org/web/packages/gutenbergr/vignettes/intro.html

In [4]:
install.packages("rlist")


The downloaded binary packages are in
	/var/folders/nw/3fnxvmmj6lb4vv4_d_jr__v40000gn/T//RtmpgJe2cc/downloaded_packages


In [1]:
library(rlist)

In [2]:
library(tm) #to process text
library(topicmodels)
library(dplyr)
library(tidytext)
library(tidyverse)
library(SnowballC) # for stemming
library(stringr)
library(ldatuning)
library(gutenbergr)
library(rlist) 

#append is functionally identical to list.append for what I want to do
#rlist's list.append() does allow you to avoid variable reassignment, though

Loading required package: NLP

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ readr   1.1.1
✔ tibble  1.4.2     ✔ purrr   0.2.5
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ ggplot2 3.0.0     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ ggplot2::annotate() masks NLP::annotate()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()


In [3]:
dim(gutenberg_metadata)

[1] 51997     8

In [4]:
#gutenberg works has no dupes, only english & books with text
dim(gutenberg_works())

[1] 40737     8

In [5]:
gutenberg_ids = NA

In [6]:
unique(gutenberg_works()$gutenberg_bookshelf)

[1] NA                                                                                                                           
  [2] "United States Law/American Revolutionary War/Politics"                                                                      
  [3] "American Revolutionary War/Politics/United States Law"                                                                      
  [4] "US Civil War"                                                                                                               
  [5] "American Revolutionary War/Politics/United States/United States Law"                                                        
  [6] "American Revolutionary War"                                                                                                 
  [7] "Children's Literature"                                                                                                      
  [8] "Children's Literature/Best Books Ever Listings"                                                                             
  [9] "CIA World Factbooks"                                                                                                        
 [10] "Best Books Ever Listings/Adventure"                                                                                         
 [11] "Christianity/Latter Day Saints"                                                                                             
 [12] "Politics"                                                                                                                   
 [13] "Native America/Children's Literature/Poetry"                                                                                
 [14] "Poetry/Banned Books from Anne Haight's list"                                                                                
 [15] "Reference"                                                                                                                  
 [16] "Slavery/African American Writers"                                                                                           
 [17] "Harvard Classics"                                                                                                           
 [18] "Harvard Classics/Best Books Ever Listings/Plays"                                                                            
 [19] "Best Books Ever Listings"                                                                                                   
 [20] "Movie Books/Harvard Classics"                                                                                               
 [21] "Science Fiction/Movie Books"                                                                                                
 [22] "Movie Books/Science Fiction"                                                                                                
 [23] "Children's Literature/Harvard Classics"                                                                                     
 [24] "Horror/Movie Books/Precursors of Science Fiction/Gothic Fiction"                                                            
 [25] "Opera"                                                                                                                      
 [26] "Christmas/Children's Literature"                                                                                            
 [27] "Mathematics"                                                                                                                
 [28] "Children's Literature/Fantasy"                                                                                              
 [29] "Harvard Classics/Philosophy"                                                                                                
 [30] "Best Books Ever Listings/Adventure/Movie Books"                                                                             
 [31] "Philosophy/Banned Books from Anne H

In [7]:
gutenberg_ids = NA

gutenberg_ids <- append(gutenberg_ids,
       gutenberg_works(
            str_detect(
                gutenberg_bookshelf, "Fantasy"))$gutenberg_id)

gutenberg_ids <- append(gutenberg_ids,
       gutenberg_works(
            str_detect(
                gutenberg_bookshelf, "Adventure"))$gutenberg_id)

gutenberg_ids <- append(gutenberg_ids,
       gutenberg_works(
            str_detect(
                gutenberg_bookshelf, "Fiction"))$gutenberg_id)

gutenberg_ids <- append(gutenberg_ids,
       gutenberg_works(
            str_detect(
                gutenberg_bookshelf, "Arthurian"))$gutenberg_id)

gutenberg_ids <- append(gutenberg_ids,
       gutenberg_works(
            str_detect(
                gutenberg_bookshelf, "Children"))$gutenberg_id)

gutenberg_ids <- append(gutenberg_ids,
       gutenberg_works(
            str_detect(
                gutenberg_bookshelf, "Myth"))$gutenberg_id)

gutenberg_ids <- append(gutenberg_ids,
       gutenberg_works(
            str_detect(
                gutenberg_bookshelf, "Folklore"))$gutenberg_id)

gutenberg_ids <- append(gutenberg_ids,
       gutenberg_works(
            str_detect(
                gutenberg_bookshelf, "Pirates"))$gutenberg_id)

gutenberg_ids<-unique(gutenberg_ids)
length(gutenberg_ids)

[1] 3537

In [8]:
books <- gutenberg_download(c(gutenberg_ids), meta_fields = c("title","author"))
# gutenberg_download(c(gutenberg_ids), meta_fields = "title")

Determining mirror for Project Gutenberg from http://www.gutenberg.org/robot/harvest
Using mirror http://aleph.gutenberg.org
Warning message in .f(.x[[i]], ...):
“Could not download a book at NA”Warning message in .f(.x[[i]], ...):
“Could not download a book at http://aleph.gutenberg.org/1/9/4/6/19467/19467.zip”Warning message in .f(.x[[i]], ...):
“Could not download a book at http://aleph.gutenberg.org/1/9/6/7/19678/19678.zip”Warning message in .f(.x[[i]], ...):
“Could not download a book at http://aleph.gutenberg.org/1/9/5/1/19513/19513.zip”Multiple files in zip: reading '25438.txt'
Warning message in .f(.x[[i]], ...):
“Could not download a book at http://aleph.gutenberg.org/4/7/6/4769/4769.zip”Multiple files in zip: reading '11757.txt'
Warning message in .f(.x[[i]], ...):
“Could not download a book at http://aleph.gutenberg.org/1/9/5/1/19514/19514.zip”Warning message in .f(.x[[i]], ...):
“Could not download a book at http://aleph.gutenberg.org/1/9/5/7/19579/19579.zip”Warning message

### Saving the dataframe in a few formats:

In [9]:
save(books,file="books.Rda")

In [11]:
# # Install feather
# devtools::install_github("wesm/feather/R")

library(feather)

In [13]:
path <- "books.feather"
write_feather(books, path)

In [ ]:
# # Encoding is latin1
write.csv(books,file="books.csv",row.names=FALSE)

# # Alternatively, this should work if I really want it in UTF-8
text2 <- iconv(books, from = "latin1", to = "UTF-8")
write.csv(text2,file="books.txt",row.names=FALSE)

In [ ]:
head(books)